In [1]:
few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.train.short.csv'
verify_few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/Food-verify.expert.train.short.csv'
data_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.smalltest.csv'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/test-results-autolabel-ner-qa'
model_name = "gpt-3.5-turbo"
label_column = 'Food_answer'
text_column = 'context'
example_selection_label_column = 'has_Food'
label_symbol = "^^^^"
few_shot_num = 8
few_shot_selection = 'label_diversity_similarity'
token_path = "/home/dzhang5/.cache/huggingface/token"
cache=False
console_output=True
temperature=0.1
verify=True
label_version='v1'
task_version='v2'
random_shuffle_examples = True
random_shuffle_examples_seed = 1

# Generation Stage

In [3]:
from autolabel.schema import ModelProvider, TaskType
from autolabel.models import register_model, MODEL_REGISTRY
from hf_pipeline_new import HFPipelineLLMNew
from few_shot_new import NewAutoLabelConfig, NewExampleSelectorFactory
from autolabel.few_shot import ExampleSelectorFactory
from template_inst import update_inst_mode
from named_entity_recognition_new import NewNamedEntityRecognitionTask
from classification_new import NewClassificationTask
from autolabel.tasks import TASK_TYPE_TO_IMPLEMENTATION 
from prompt_template import load_ner_task_prompt
import sys

In [4]:
update_inst_mode(model_name)
TASK_TYPE_TO_IMPLEMENTATION[TaskType.NAMED_ENTITY_RECOGNITION] = NewNamedEntityRecognitionTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.CLASSIFICATION] = NewClassificationTask
sys.modules['autolabel.labeler'].ExampleSelectorFactory = NewExampleSelectorFactory
register_model(ModelProvider.HUGGINGFACE_PIPELINE, HFPipelineLLMNew)

In [5]:
from autolabel import LabelingAgent, AutolabelDataset
import json
import os
import pandas as pd

In [6]:
with open(token_path) as tfile:
    token_str = tfile.read()

from huggingface_hub import login
login(token=token_str)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dzhang5/.cache/huggingface/token
Login successful


In [7]:
no_auto = ["microsoft/prophetnet", "microsoft/phi-2", "google/pegasus-x"]
if any([pre in model_name for pre in no_auto]):
    device_map = None
else:
    device_map = "auto"

In [8]:
if not os.path.exists(output_dir):
    # Create the directory
    os.makedirs(output_dir)
label_type = label_column.split('_')[0]
output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_' + label_type + '_' + os.path.split(data_path)[-1]
output_path = os.path.join(output_dir, output_name)

In [ ]:
ori_output = pd.read_csv(output_path)

In [9]:
refuel_models = ["refuel-llm", "llama-13b-chat"]
if model_name in refuel_models:
    provider = "refuel"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetNERQA_{few_shot_selection}_{label_type}_{model_name}"
elif model_name.startswith('gpt'):
    provider = "openai"
    em_provider = "openai"
    model_params = {"max_tokens":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetNERQA_{few_shot_selection}_{label_type}_{model_name}"
else:
    provider = "huggingface_pipeline"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature,
                    "quantize": 16, "device_map": device_map,
                    "token": token_str}
    task_name = f"FoodborneIllnessIncidentTweetNERQA_{few_shot_selection}_{label_type}_{model_name.split('/')[1]}"

In [ ]:
sym_len = len(label_symbol)
label_prefix, label_suffix, label_description, task_guideline, output_guideline = load_ner_task_prompt(label_type, label_symbol, label_version, task_version, False)

In [11]:
test_data = pd.read_csv(data_path)

In [ ]:
assert (test_data.index == ori_output.index).all()
assert (test_data[text_column] == ori_output[text_column]).all()

In [ ]:
relabel_test_data = test_data.loc[ori_output[f'{label_column}_error'].notna()].copy()

In [ ]:
config = {
    "task_name": task_name,
    "task_type": "question_answering",
    "dataset": {
        "label_column": label_column,
        "text_column": text_column,
        "example_selection_label_column": example_selection_label_column,
        "delimiter": ",",
        "label_description": label_description 
    },
    "model": {
        "provider": provider,
        "name": model_name,
        "params": model_params
    },
    "embedding": {
        "provider": em_provider,
    },
    "prompt": {
        "task_guidelines": task_guideline,
        "output_guidelines": output_guideline,
        "labels": [
            "Food",
            "Symptom",
            "Location",
            "Keyword"
        ],
        "few_shot_examples": few_shot_path,
        "few_shot_selection": few_shot_selection,
        "few_shot_num": few_shot_num,
        "random_shuffle_examples": random_shuffle_examples,
        "random_shuffle_examples_seed": random_shuffle_examples_seed,
        "example_template": f"Input: {{{text_column}}}\nOutput: {{{label_column}}}"
    }
}

config = NewAutoLabelConfig(config)

In [ ]:
agent = LabelingAgent(config=config, console_output=console_output, cache=cache)

2024-03-12 18:18:28 autolabel.labeler WARNING: cache parameter is deprecated and will be removed soon. Please use generation_cache and transform_cache instead.
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
ds = AutolabelDataset(test_data[[relabel_text_column, example_selection_label_column, label_column]], config=config)

In [ ]:
agent.plan(ds)

/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
2024-03-12 18:18:30 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Output()

2024-03-12 18:18:32 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-12 18:18:32 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-12 18:18:32 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-12 18:18:33 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-12 18:18:33 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-12 18:18:33 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-12 18:18:33 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-12 18:18:33 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-12 18:18:33 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-12 18:18:34 httpx INFO: HTTP Request: 

┌──────────────────────────┬──────────┐
│ Total Estimated Cost     │ $34.2387 │
│ Number of Examples       │ 3710     │
│ Average cost per example │ $0.0092  │
└──────────────────────────┴──────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at extracting Symptom entities that are related to foodborne illness incident from text. In the 
given text, your task is to label Symptom entities that are specific symptoms experienced by the affected person as
a result of the suspected foodborne illness.. Note that you should surround the extracted entities in the text with
"^^" and "^^". Use the following examples as a guide for your predictions and format your responses similarly.

You will answer with just the correct labeled sentence and nothing else. Note that if the given text does not 
include any Symptom entity related to foodborne illness, you will answer with the input text.

Some examples with their output answers are provided below:

Input: I'm not sending this my stomach again . . . . I'm wearing these fire outfits [ 
EMOJI_smiling_face_with_heart-eyes ] [ EMOJI_smiling_face_with_heart-eyes ]
Output: I'm not sending this my stomach again . . . . I'm wearing these fire outfits [ 
EMOJI_smiling_face_with_heart-eyes ] [ EMOJI_smiling_face_with_heart-eyes ]

Input: How you gone say stop kicking my stomach but you fighting while pregnant . . . . . . [ EMOJI_thinking_face ]
[ EMOJI_thinking_face ] [ EMOJI_thinking_face ]
Output: How you gone say stop kicking my stomach but you fighting while pregnant . . . . . . [ EMOJI_thinking_face 
] [ EMOJI_thinking_face ] [ EMOJI_thinking_face ]

Input: I'm fina punch my stomach so hard [ EMOJI_loudly_crying_face ] Bc why tf is it hurting .
Output: I'm fina punch my stomach so hard [ EMOJI_loudly_crying_face ] Bc why tf is it hurting .

Input: Really hope I don't have a stomach bug or food poisoning [ EMOJI_disappointed_face ]
Output: Really hope I don't have a stomach bug or food poisoning [ EMOJI_disappointed_face ]

Input: Watching The abduction of Lisa Mcvey : Believe me and my stomach is in absolute knots [ EMOJI_crying_face ] 
that poor poor girl [ EMOJI_loudly_crying_face ] [ EMOJI_loudly_crying_face ]
Output: Watching The abduction of Lisa Mcvey : Believe me and my stomach is in absolute knots [ EMOJI_crying_face ]
that poor poor girl [ EMOJI_loudly_crying_face ] [ EMOJI_loudly_crying_face ]

Input: @USER @USER @USER when i was younger i knew a girl that had to get reconstructive surgery on her nose 
multiple times cause her mom got kicked in the stomach fighting while pregnant [ EMOJI_confused_face ] jail !
Output: @USER @USER @USER when i was younger i knew a girl that had to get reconstructive surgery on her nose 
multiple times cause her mom got kicked in the stomach fighting while pregnant [ EMOJI_confused_face ] jail !

Input: my heart aches all the time from how much i wuv you and when youre there i feel like ill puke my heart out [
EMOJI_angry_face ] [ EMOJI_angry_face ] [ EMOJI_angry_face ]
Output: my heart aches all the time from how much i wuv you and when youre there i feel like ill puke my heart out 
[ EMOJI_angry_face ] [ EMOJI_angry_face ] [ EMOJI_angry_face ]

Input: I cannot believe I've got food poisoning 2 days before I go on holiday [ EMOJI_upside-down_face ]
Output: I cannot believe I've got food poisoning 2 days before I go on holiday [ EMOJI_upside-down_face ]

Now I want you to label the following example:
Input: I know my tubes tied but When Mother Nature be playing I give my stomach a extra punch just in case cause 
bitch don't play with me [ EMOJI_face_with_raised_eyebrow ] [ EMOJI_unamused_face ] [ EMOJI_loudly_crying_face ]
Output:

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
# now, do the actual labeling
ds = agent.run(ds)

In [ ]:
metrics = ds.eval()

In [ ]:
new_output = ds.df.copy()

In [ ]:
new_output.index = ori_output.loc[ori_output[f'{label_column}_error'].notna()].index

In [ ]:
ori_output.loc[ori_output[f'{label_column}_error'].notna()] = new_output

In [ ]:
ori_output.to_csv(output_path, index=False)
ori_output.to_pickle(output_path.replace('.csv', '.pkl'))

In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        return []
    
if not verify:
    raise StopExecution

# Verification Stage

In [ ]:
from utils import get_predictions, check_prediction, get_verify_df, construct_results

In [ ]:
verify_question_column = f'{label_type}_verify_question'
verify_answer_column = f'{label_type}_verify_answer'
pos_column = f'{label_column}_pos'

In [ ]:
test_data[[verify_question_column, pos_column]] = ds.df[[f'{label_column}_label']].apply(lambda x: get_predictions(x[f'{label_column}_label'], label_symbol), axis=1,result_type="expand")

In [ ]:
test_data[verify_answer_column] = test_data[['CategorizedLabels', verify_question_column]].apply(lambda x: check_prediction(x['CategorizedLabels'], x[verify_question_column], label_type), axis=1)

In [ ]:
new_test_data = get_verify_df(test_data, verify_question_column, verify_answer_column, text_column, pos_column)

In [ ]:
verify_task_guideline = (f"You are an expert at identifying {label_type} entities that are related to foodborne illness incident from text. " 
                         f"In the given text, your task is to verify if a given word is a {label_type} entity that {label_description[label_type].replace('are ', 'is ').replace('.', '')} in the given text. "
                         "Use the following examples as a guide for your analysis and format your responses similarly.")
question = f'''Do you think the word "{{{verify_question_column}}}" in the given text is a {label_type} entity that {label_description[label_type].replace('are ', 'is ').replace('.', '')}?'''

In [ ]:
output_verify_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_' + label_type + '_verify_' + os.path.split(data_path)[-1]
output_verify_path = os.path.join(output_dir, output_verify_name)
output_final_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_' + label_type + '_final_' + os.path.split(data_path)[-1]
output_final_path = os.path.join(output_dir, output_final_name)

In [ ]:
config = {
    "task_name": task_name+'_verification',
    "task_type": "classification",
    "dataset": {
        "label_column": verify_answer_column,
        "text_column": text_column,
        "delimiter": ",",
        "label_description": label_description 
    },
    "model": {
        "provider": em_provider,
        "name": model_name,
        "params": model_params
    },
    "embedding": {
        "provider": em_provider,
    },
    "prompt": {
        "task_guidelines": verify_task_guideline,
        "output_guidelines": f'''You will answer with just the correct output label ("Yes" or "No") and nothing else.''',
        "labels": [
            "Yes",
            "No"
        ],
        "few_shot_examples": verify_few_shot_path,
        "few_shot_selection": few_shot_selection,
        "few_shot_num": few_shot_num,
        "random_shuffle_examples": random_shuffle_examples,
        "random_shuffle_examples_seed": random_shuffle_examples_seed,
        "example_template": f"Context: {{{text_column}}}\nQuestion: {question}\nAnswer: {{{verify_answer_column}}}"
    }
}

config = NewAutoLabelConfig(config)

In [ ]:
agent = LabelingAgent(config=config, console_output=console_output, cache=cache)

In [ ]:
verify_ds = AutolabelDataset(new_test_data.drop(['text_idx',pos_column], axis=1), config=config)

In [ ]:
agent.plan(verify_ds)

In [ ]:
# now, do the actual labeling
verify_ds = agent.run(verify_ds)

In [ ]:
metrics = verify_ds.eval()

In [ ]:
new_test_data[f'{verify_answer_column}_label'] = verify_ds.df[f'{verify_answer_column}_label']

In [ ]:
verify_ds.df['text_idx'] = new_test_data['text_idx']

In [ ]:
verify_ds.df.to_csv(output_verify_path, index=False)
verify_ds.df.to_pickle(output_verify_path.replace('.csv', '.pkl'))

In [ ]:
new_result = construct_results(ds.df, new_test_data, f'{label_column}_label', f'{verify_answer_column}_label', "text_idx", pos_column, label_symbol)

In [ ]:
metrics = ds.eval()

In [ ]:
new_result.to_csv(output_final_path, index=False)
new_result.to_pickle(output_final_path.replace('.csv', '.pkl'))